# Word2Vec
___
## Notes
In this notebook, we will discuss the word2vec algorithm.

Specifically, we break our discussion down into the following sections:
> [Overview](#Overview:-word2vec)
>
> [Implementation](#Implementation)
>

We finish the notebook off with a [review](#Review) of everything discussed. 